# Using cottoncandy to access the Allen Brain Observatory data from AWS S3

This examples shows how to use **cottoncandy** to access the data provided by the Allen Brain Observatory.

This notebook is based on [this example](https://github.com/AllenInstitute/AllenSDK/wiki/Use-the-Allen-Brain-Observatory-%E2%80%93-Visual-Coding-on-AWS) provided by the Allen Institute.

To find out more about cottoncandy, checkout our GitHub repo: 
https://github.com/gallantlab/cottoncandy

---
*Contributed by:*
Anwar O Nunez-Elizalde (Aug, 2018)


In [1]:
!pip install cottoncandy

## Setting up the cottoncandy connection

In order to run this example, you will need to enter your AWS keys below

In [0]:
ACCESSKEY = 'FAKEAK' 
SECRETKEY = 'FAKESK'

In [3]:
import cottoncandy as cc
cci = cc.get_interface('allen-brain-observatory', ACCESS_KEY=ACCESSKEY, SECRET_KEY=SECRETKEY, endpoint_url='https://s3.amazonaws.com')

Available buckets:
cottoncandy-test                         2018/08/10 (01:18:55)
Current bucket: allen-brain-observatory


## Browsing the data contents

In [4]:
cci.lsdir()

[u'visual-coding-2p']

In [5]:
cci.lsdir('visual-coding-2p')

[u'visual-coding-2p/ophys_experiment_analysis',
 u'visual-coding-2p/ophys_experiment_data',
 u'visual-coding-2p/ophys_movies',
 u'visual-coding-2p',
 u'visual-coding-2p/cell_specimens.json',
 u'visual-coding-2p/experiment_containers.json',
 u'visual-coding-2p/manifest.json',
 u'visual-coding-2p/ophys_experiments.json',
 u'visual-coding-2p/stimulus_mappings.json']

In [6]:
experiment_data = cci.download_json('visual-coding-2p/ophys_experiments.json')
print('Number of available experiments: %i'%len(experiment_data))

Number of available experiments: 648


In [7]:
cci.search('visual-coding-2p/ophys_movies/*')

visual-coding-2p/ophys_movies/ophys_experiment_496908818.h5    2018/06/05 (07:56:44) 56933.6M
visual-coding-2p/ophys_movies/ophys_experiment_496934409.h5    2018/06/05 (07:56:44) 52854.7M
visual-coding-2p/ophys_movies/ophys_experiment_496935917.h5    2018/06/05 (07:56:44) 52853.2M
visual-coding-2p/ophys_movies/ophys_experiment_497060401.h5    2018/06/05 (07:56:44) 52977.2M
visual-coding-2p/ophys_movies/ophys_experiment_497256116.h5    2018/06/05 (07:56:44) 57058.6M
visual-coding-2p/ophys_movies/ophys_experiment_500855614.h5    2018/06/05 (07:56:44) 52985.2M
visual-coding-2p/ophys_movies/ophys_experiment_500860585.h5    2018/06/05 (07:56:44) 56929.1M
visual-coding-2p/ophys_movies/ophys_experiment_500947031.h5    2018/06/05 (07:56:44) 57069.1M
visual-coding-2p/ophys_movies/ophys_experiment_500964514.h5    2018/06/05 (07:56:44) 57054.6M
visual-coding-2p/ophys_movies/ophys_experiment_501004031.h5    2018/06/05 (07:56:44) 57742.7M
visual-coding-2p/ophys_movies/ophys_experiment_501021421.h5 

In [8]:
sample_experiment = experiment_data[0]
print('Experiment ID: %i'%sample_experiment['id'])
cci.search('visual-coding-2p/ophys_movies/ophys_experiment_%d*'%sample_experiment['id'])

Experiment ID: 511458874
visual-coding-2p/ophys_movies/ophys_experiment_511458874.h5    2018/06/05 (09:04:53) 56928.6M
Found 1 objects matching "visual-coding-2p/ophys_movies/ophys_experiment_511458874*"


## Downloading the data

Below, we download the HDF5 data locally to disk. We then use `h5py` to load the data into python.
This is inefficient because the data is downloaded to disk and then read into memory into the Jupyter session.

**WARNING:** The data is 50GB in size! This will take a while to download

In [0]:
local_destination_path = '/tmp/brain_data.hdf5'
_ = cci.download_to_file('visual-coding-2p/ophys_movies/ophys_experiment_%d.h5'%sample_experiment['id'], '/tmp/brain_data.hdf5')

## Plotting stuff

In [0]:
import h5py
from matplotlib import pyplot as plt

with h5py.File(local_destination_path,'r') as hfl:
  frames = hfl["data"][:10,:,:]
  plt.imshow(frames[0])
  plt.show()
